In [1]:
# QMR execution settings
plot_sne = False                         # t-SNE plot? (requires a bit of RAM)
plot_metrics_comp = False                 # metrics comparison?
savefig = False                           # save figs or show in line
use_existing_metrics = True              # read existing metrics output data files instead of processing them?
regressor_quality_metrics = ['sigma','snr','rer','sharpness','scale','score']

#Define path of the original (reference) datasets
data_paths = {
    "DeepGlobe469": "./Data/DeepGlobe",
    "UCMerced2100": "./Data/UCMerced_LandUse",
    "UCMerced380": "./Data/test-ds",
    "USGS279": "./Data/USGS",
    "shipsnet-ships4000": "./Data/shipsnet",
    "shipsnet-scenes7": "./Data/shipsnet",
    "inria-test10": "./Data/inria-aid_short",
    "inria-test180": "./Data/AerialImageDataset",
    "inria-train180": "./Data/AerialImageDataset",
    "XView-train846": "./Data/XView",
    "XView-val281": "./Data/XView",
}
image_folders = {
    "DeepGlobe469": "images",
    "UCMerced2100": "Images/ALL_CATEGORIES",
    "UCMerced380": "test",
    "USGS279": "hr_images",
    "shipsnet-ships4000": "shipsnet",
    "shipsnet-scenes7": "scenes/scenes",
    "inria-test10": "test/images_short",
    "inria-test180": "test/images",
    "inria-train180": "train/images",
    "XView-train846": "train_images",
    "XView-val281": "val_images",
}

In [2]:
# Imports
import os
import argparse
import shutil
import mlflow
import pandas as pd
from pdb import set_trace as debug # debugging

# display tables of max 50 columns
pd.set_option('display.max_columns', 50)

from custom_modifiers import DSModifierFake
from iquaflow.datasets import DSModifier, DSWrapper
from iquaflow.experiments import ExperimentInfo, ExperimentSetup
from iquaflow.experiments.experiment_visual import ExperimentVisual
from iquaflow.experiments.task_execution import PythonScriptTaskExecution
from iquaflow.quality_metrics import RERMetrics, SNRMetrics, GaussianBlurMetrics, NoiseSharpnessMetrics, GSDMetrics, ScoreMetrics
from visual_comparison import metric_comp, plotSNE

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  """


In [3]:
# Remove previous mlflow records of previous executions of the same experiment
try: # rm_experiment
    mlflow.delete_experiment(ExperimentInfo(f"{experiment_name}").experiment_id)
    # Clean mlruns and __pycache__ folders
    shutil.rmtree("mlruns/",ignore_errors=True)
    os.makedirs("mlruns/.trash", exist_ok=True)
    shutil.rmtree(f"{data_path}/.ipynb_checkpoints",ignore_errors=True)
    [shutil.rmtree(x) for x in glob(os.path.join(os.getcwd(), "**", '__pycache__'), recursive=True)]
except:
    pass


In [4]:
dataframes = []
for ids, database_name in enumerate(list(data_paths.keys())):

    data_path = data_paths[database_name]
    images_path = os.path.join(data_paths[database_name],image_folders[database_name])
    python_ml_script_path = 'sr.py'  

    #Define name of IQF experiment
    experiment_name='eoqmr'
    experiment_name += f"_{database_name}"
    
    # set output folders
    plots_folder = "plots/"+experiment_name+"/"
    results_folder = "results/"+experiment_name+"/"
    os.makedirs(plots_folder, exist_ok=True)
    os.makedirs(results_folder, exist_ok=True)
    
    # plot SNE of existing images
    if plot_sne:
        plotSNE(database_name, data_path, (232,232), 6e4, True, savefig, plots_folder)

    #DS wrapper is the class that encapsulate a dataset
    ds_wrapper = DSWrapper(data_path=data_path)
    
    # Define and execute script (sr.py: copy image files to testing folder)
    ds_wrapper = DSWrapper(data_path=data_path)
    ds_modifiers_list = [DSModifier()] # DSModifierFake(name="base",images_dir=images_path)
    task = PythonScriptTaskExecution( model_script_path = python_ml_script_path )
    experiment = ExperimentSetup(
        experiment_name=experiment_name,
        task_instance=task,
        ref_dsw_train=ds_wrapper,
        ds_modifiers_list=ds_modifiers_list,
        ref_dsw_val=ds_wrapper,
        repetitions=1,
        extra_train_params={"trainds": [data_path], "traindsinput": [images_path], "valds": [data_path],"valdsinput": [images_path]}
    )
    # Enforce replacement of paths in experiment wrappers
    '''
    experiment.ref_dsw_train.data_input = images_path
    if experiment.ref_dsw_test is not None:
        experiment.ref_dsw_test.data_input = images_path
    if experiment.ref_dsw_val is not None:
        experiment.ref_dsw_val.data_input = images_path 
    '''

    #Execute the experiment
    experiment.execute()
    experiment_info = ExperimentInfo(experiment_name)
    # ExperimentInfo is used to retrieve all the information of the whole experiment. 
    # It contains built in operations but also it can be used to retrieve raw data for futher analysis

    print('Calculating Quality Metric Regression...'+",".join(regressor_quality_metrics)) #default configurations
    path_regressor_quality_metrics = f'./{results_folder}regressor_quality_metrics.csv'
    if use_existing_metrics and os.path.exists(path_regressor_quality_metrics):
        df = pd.read_csv(path_regressor_quality_metrics, index_col=None)
    else:
        _ = experiment_info.apply_metric_per_run(ScoreMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(RERMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(SNRMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(GaussianBlurMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(NoiseSharpnessMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(GSDMetrics(), ds_wrapper.json_annotations)
        df = experiment_info.get_df(
            ds_params=["ds_modifier"],
            metrics=regressor_quality_metrics,
            dropna=False
        )
        df.to_csv(path_regressor_quality_metrics)

    # check results
    df["ds_modifier"] = database_name
    
    print(f"writing {path_regressor_quality_metrics}")
    df.to_csv(path_regressor_quality_metrics)
    
    # append table of current dataset
    dataframes.append(df)


Directory not copied. Error: [Errno 17] File exists: './Data/DeepGlobe#base_modifier/images'
Directory not copied. Error: [Errno 17] File exists: './Data/DeepGlobe#base_modifier/images'
python sr.py --trainds ./Data/DeepGlobe#base_modifier --outputpath /tmp/tmpqig3ttqq --trainds ./Data/DeepGlobe --traindsinput ./Data/DeepGlobe/images --valds ./Data/DeepGlobe#base_modifier --valdsinput ./Data/DeepGlobe/images
****** IQF subprocess --stdout-- *********

****** IQF subprocess --stderr-- *********

Calculating Quality Metric Regression...sigma,snr,rer,sharpness,scale,score
writing ./results/eoqmr_DeepGlobe469/regressor_quality_metrics.csv
Directory not copied. Error: [Errno 17] File exists: './Data/UCMerced_LandUse#base_modifier/Images'
Directory not copied. Error: [Errno 17] File exists: './Data/UCMerced_LandUse#base_modifier/Images'
python sr.py --trainds ./Data/UCMerced_LandUse#base_modifier --outputpath /tmp/tmp_bgan586 --trainds ./Data/UCMerced_LandUse --traindsinput ./Data/UCMerced_L

./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (11/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (12/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (13/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (14/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (15/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (16/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (17/32) for 19tdl080220
./Data/USGS/hr_images/19tdl080220.tif
torch.Size

./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (8/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (9/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (10/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (11/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (12/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (13/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (14/32) for 14skc085155
./Data/USGS/hr_images/14skc085155.tif
torch.Size([

./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (4/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (5/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (6/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (7/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (8/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (9/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (10/32) for 13uep490790
./Data/USGS/hr_images/13uep490790.tif
torch.Size([1, 3

./Data/USGS/hr_images/11t_lp_945130.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (32/32) for 11t_lp_945130
./Data/USGS/hr_images/11t_lp_945130.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (1/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (2/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (3/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (4/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (5/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size([1, 3, 5000, 5000])
torch.Size([1, 3, 32, 32])
Generating crop (6/32) for 14spb375285
./Data/USGS/hr_images/14spb375285.tif
torch.Size

...

(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop17.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop18.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop19.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop20.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop21.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop22.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop23.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/14tlq840275_crop24.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_3

(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop2.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop3.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop4.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop5.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop6.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop7.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop8.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/13uep490790_crop9.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32

(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop27.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop28.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop29.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop30.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop31.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/11t_lp_945130_crop32.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
deploying ['sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigma', 'sigm

torch.Size([3, 1836, 1130])
torch.Size([3, 1836, 1130])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop19.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop20.png
(32, 2369)
torch.Size([3, 2369, 32])
torch.Size([3, 2369, 32])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop21.png
(670, 1104)
torch.Size([3, 1104, 670])
torch.Size([3, 1104, 670])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop22.png
(32, 883)
torch.Size([3, 883, 32])
torch.Size([3, 883, 32])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop23.png
(32, 32)
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop24.png
(32, 1937)
torch.Size([3, 1937, 32])
torch.Size([3, 1937, 32])
./Data/USGS/hr_images_32crops32x32/c_12001384_12_15100_4bd_2014_crop25.png
(633, 32)
torch.Size([3, 32, 633])
torch.Size([3, 32,

RuntimeError: stack expects each tensor to be equal size, but got [3, 2048, 657] at entry 0 and [3, 32, 1458] at entry 1

In [ ]:
# concat all dataset tables
#df = pd.concat(dataframes)
results_folder = "results/"
path_all_datasets = f"{results_folder}results.csv"
print(f"writing csv: {path_all_datasets}")
df = pd.concat(dataframes,axis=0)
{df.drop(str(field),inplace=True,axis=1) for field in df if "Unnamed" in str(field)}
df.to_csv(path_all_datasets)

In [ ]:
df

In [ ]:
# plot metric comparison
if plot_metrics_comp:
    metric_comp(df,regressor_quality_metrics,savefig,plots_folder)